In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
de_id_data = '../../derivatives/phenotypic/deidentified_phenotypic_data.csv'
pheno_all = pd.read_csv(de_id_data,index_col=0)
data_dict_raw = '../../derivatives/phenotypic/Voice_DataDictionary_2022-02-25.csv'

col_to_cat = pd.read_csv(data_dict_raw, encoding="iso-8859-1") #do file -I <file> in a terminal to determine the encoding type since it threw and error 
col_to_cat.rename(columns={'Variable / Field Name': 'Variable'}, inplace=True) #first column is named oddly w/ the encoding, rename to useful name
col_to_cat = col_to_cat[col_to_cat.Variable != 'qidssr_instructions'] #apparently this label is in the dictionary but not in the data. It is just the instructions so it was probably accidental
col_to_cat = col_to_cat[['Variable', 'Form Name']] #only care about these right now. Field Label could be interesting to map from the survey form to variable

In [49]:
#data has a bunch of test voice ID so we just want ones from bids
valid_voice_id = [name.split('sub-voice')[1] for name in glob.glob('../../sub-voice*')]
pheno_all = pheno_all[pheno_all.voice_id.isin(valid_voice_id)]

In [12]:
pheno_all[pheno_all.voice_id == '955'][['voice_id', 'sex', 'age']]

,voice_id,sex,age
109,955,0.0,27.0
115,955,1.0,53.0


In [131]:

demographics = ['voice_id','age','sex'] #1 is male
#just get demographics and raw BDI scores
bdi_scores = pheno_all[pd.concat([pd.Series(demographics), data_dict['beckdepressionii'].Variable])]

#drop existing total because it did not calculate for all
bdi_scores = bdi_scores.drop('beckdepressionii_total', axis=1)

#calcualte bdi scores
bdi_scores['beckdepressionii_total'] = bdi_scores.drop(demographics, axis=1).sum(axis=1)

In [132]:
bdi_scores_drop_na = bdi_scores.dropna()

In [133]:
bdi_scores_drop_na

,voice_id,age,sex,beckdepressionii_1,beckdepressionii_2,beckdepressionii_3,beckdepressionii_4,beckdepressionii_5,beckdepressionii_6,beckdepressionii_7,...,beckdepressionii_13,beckdepressionii_14,beckdepressionii_15,beckdepressionii_16,beckdepressionii_17,beckdepressionii_18,beckdepressionii_19,beckdepressionii_20,beckdepressionii_21,beckdepressionii_total
9,875,65.0,1.0,2.0,3.0,2.0,3.0,0.0,0.0,3.0,...,2.0,2.0,1.0,3.0,1.0,3.0,1.0,2.0,3.0,38.0
10,873,51.0,1.0,1.0,1.0,3.0,2.0,1.0,3.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,38.0
11,872,47.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,2.0,...,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,29.0
15,886,23.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,...,3.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,1.0,36.0
17,893,24.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,3.0,2.0,2.0,1.0,0.0,2.0,2.0,2.0,2.0,35.0
19,891,25.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,...,0.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,1.0,24.0
21,889,62.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,20.0
25,898,20.0,0.0,2.0,1.0,1.0,2.0,1.0,0.0,2.0,...,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,32.0
27,897,32.0,0.0,1.0,2.0,3.0,0.0,2.0,0.0,2.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
28,896,27.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,1.0,...,0.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,20.0


In [135]:
bdi_scores_drop_na.to_csv('bdi_scores.csv')

In [136]:
subj_missing_bdi = bdi_scores[~bdi_scores.index.isin(bdi_scores_drop_na.index)]
subj_missing_bdi.voice_id

47    999
49    998
50    997
51    996
52    995
53    994
54    993
55    990
78    991
79    992
80    989
81    988
82    987
Name: voice_id, dtype: object

In [114]:
pd.options.display.max_rows=20

In [122]:
bdi_scores_drop_na['age'] = pheno_all.age
bdi_scores_drop_na['sex'] = pheno_all.sex

/cm/shared/openmind/anaconda/3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/cm/shared/openmind/anaconda/3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [117]:
pheno_all[~bdi_scores.index.isin(bdi_scores_drop_na.index)].beckdepressionii_complete

47    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
78    0
79    0
80    0
81    0
82    0
Name: beckdepressionii_complete, dtype: int64

In [26]:
#get column names with beck scores in it 
#and remove non-useful colums: timestamp, complete flag, and total since it wasn't always correct
# bdi_col = [p for p in pheno_all.columns 
#            if 'beckdepression' in p
#            if 'timestamp' not in p 
#            if 'total' not in p 
#            if 'complete' not in p]


#pull answers to bdi questionare for each valid participant
#bdi_scores = pheno_all.query(f'voice_id.isin({valid_voice_id})')[['voice_id']+ bdi_col]

#calculate BDI score for each participant, totalling NaNs (blanks) as NaN
bdi_scores['beckdepressionii_score'] = bdi_scores[bdi_col].sum(axis=1, min_count=1)
bdi_scores

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [107]:
any(bdi_scores.loc[82,:].isnull())

True

In [48]:
fields = col_to_cat.groupby('Form Name')

vid = pd.Series('voice_id')

data_dict = {field_name: variable for field_name, variable in fields}



In [24]:
# split_data = {}

# # is there a cleaner way? created separarate DFs by data category i can now save!

# for fn, v in fields:
#     features = pd.concat([vid,v.Variable]) # clunky way to concatenate voice id for each row with the column values we want
#     split_data[fn] = pheno_all.query(f'voice_id.isin({valid_voice_id})')[features]

In [28]:
mri_col=[p for p in pheno_all.columns 
           if 'mri' in p]
           #if 'timestamp' not in p 
           #if 'total' not in p 
           #if 'complete' not in p]
            
pheno_all.query(f'voice_id.isin({valid_voice_id})')[['voice_id']+ mri_col]

,voice_id,mri_screening_timestamp,mri_screening_1,mri_screening_2,mri_screening_3,mri_screening_4,mri_screening_5,mri_screening_6,mri_screening_7,mri_screening_8,...,mri_screening_18,mri_screening_19,mri_screening_20,mri_screening_21,mri_screening_22,mri_screening_complete,mri_session001,mri_session002,mri_session003,mri_session004
6,876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
8,874,2015-06-11 15:24:19,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,2,NaN,NaN,NaN,NaN
9,875,2015-06-12 10:40:47,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,partial removable denture,2,NaN,NaN,NaN,NaN
10,873,2015-06-12 14:19:01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,rmovable teeth,2,NaN,NaN,NaN,NaN
11,872,2015-06-21 17:37:37,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,953,2017-09-11 11:30:44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,2,NaN,NaN,NaN,NaN
118,952,2017-09-11 14:53:37,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,small black tattoo on the leg,2,NaN,NaN,NaN,NaN
119,951,2017-10-17 16:18:52,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,2,NaN,NaN,NaN,NaN
120,950,2017-11-03 13:37:44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,2,NaN,NaN,NaN,NaN


,voice_id,beckdepressionii_1,beckdepressionii_2,beckdepressionii_3,beckdepressionii_4,beckdepressionii_5,beckdepressionii_6,beckdepressionii_7,beckdepressionii_8,beckdepressionii_9,...,beckdepressionii_13,beckdepressionii_14,beckdepressionii_15,beckdepressionii_16,beckdepressionii_17,beckdepressionii_18,beckdepressionii_19,beckdepressionii_20,beckdepressionii_21,beckdepressionii_total
9,875,2.0,3.0,2.0,3.0,0.0,0.0,3.0,2.0,0.0,...,2.0,2.0,1.0,3.0,1.0,3.0,1.0,2.0,3.0,38.0
10,873,1.0,1.0,3.0,2.0,1.0,3.0,2.0,1.0,0.0,...,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,38.0
11,872,1.0,2.0,1.0,1.0,2.0,0.0,2.0,2.0,0.0,...,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,29.0
15,886,1.0,2.0,1.0,2.0,1.0,0.0,2.0,2.0,0.0,...,3.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,1.0,36.0
17,893,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,...,3.0,2.0,2.0,1.0,0.0,2.0,2.0,2.0,2.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
119,951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
pheno_all.columns.drop(bdi_col+mri_col+['voice_id'])[:50]

Index(['study_id', 'redcap_event_name', 'redcap_survey_identifier',
       'initial_screening_timestamp', 'mobile_id', 'date_enrolled',
       'first_name', 'last_name', 'address', 'telephone_1', 'telephone_2',
       'email', 'sex', 'dob', 'age', 'weight', 'live_in_boston_area',
       'mental_illness', 'mental_illness_text', 'handed', 'metal',
       'metal_text', 'occupation', 'medication_general',
       'medication_psychological', 'language_nativeenglish',
       'language_fluencyenglish', 'depression_history_1',
       'depression_history_2', 'depression_history_3', 'drug_history_1',
       'drug_history_2', 'tbi_history_1', 'tbi_history_text',
       'seizure_history_1', 'seizure_history_text', 'hypothyroidism_history_1',
       'initial_screening_complete', 'beckdepressionii_timestamp',
       'beckdepressionii_total', 'beckdepressionii_complete',
       'minicog_timestamp', 'minicog_1', 'minicog_complete', 'mdq_timestamp',
       'mdq1a', 'mdq1b', 'mdq1c', 'mdq1d', 'mdq1e'],
 

In [13]:
# we can see that the missing BDI scores did not appear to complete the test, though not sure if this was a manually entered flag
pheno_all.query(f'voice_id.isin({valid_voice_id})')[['voice_id','beckdepressionii_complete']].head(30)

,voice_id,beckdepressionii_complete
6,876,0
8,874,2
9,875,2
10,873,2
11,872,2
12,871,2
13,870,2
14,883,2
15,886,2
16,888,2


In [99]:
#list subjects missing bdi score  - determined if bdi score is NA
 = bdi_scores.voice_id[bdi_scores.beckdepressionii_score.isna()]
subj_missing_bdi #from 

AttributeError: 'DataFrame' object has no attribute 'beckdepressionii_score'

In [27]:
bdi_col

['beckdepressionii_1',
 'beckdepressionii_2',
 'beckdepressionii_3',
 'beckdepressionii_4',
 'beckdepressionii_5',
 'beckdepressionii_6',
 'beckdepressionii_7',
 'beckdepressionii_8',
 'beckdepressionii_9',
 'beckdepressionii_10',
 'beckdepressionii_11',
 'beckdepressionii_12',
 'beckdepressionii_13',
 'beckdepressionii_14',
 'beckdepressionii_15',
 'beckdepressionii_16',
 'beckdepressionii_17',
 'beckdepressionii_18',
 'beckdepressionii_19',
 'beckdepressionii_20',
 'beckdepressionii_21']

In [69]:
test = data_dict['beckdepressionii']

In [50]:
data_dict.keys()

dict_keys(['beckanxiety', 'beckdepressionii', 'blackdogbipolar', 'generalizedanxietydisorder7', 'initial_screening', 'kbit2', 'lyubomirsky_happiness4q', 'mdq', 'minicog', 'miscnotes', 'mri_screening', 'obsessivecompulsivedisorder_anxietydepressionassociationamerica', 'patient_health_questionnaire_9', 'qidssr16', 'sage', 'scandates', 'snaithhamiltonpleasurescale', 'yale_prime_schizophrenia'])

,Variable,Form Name
35,beckdepressionii_1,beckdepressionii
36,beckdepressionii_2,beckdepressionii
37,beckdepressionii_3,beckdepressionii
38,beckdepressionii_4,beckdepressionii
39,beckdepressionii_5,beckdepressionii
40,beckdepressionii_6,beckdepressionii
41,beckdepressionii_7,beckdepressionii
42,beckdepressionii_8,beckdepressionii
43,beckdepressionii_9,beckdepressionii
44,beckdepressionii_10,beckdepressionii
